In [1]:
from utils import *
import torch

set_seed(17)

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
batch_size = 4096
window_size = 100
epochs = 20
hidden_size = 10 * window_size
num_features = 51

In [5]:
train_loader, validation_loader, columns, mm = get_train_val_dataloaders(batch_size, window_size, device)

In [6]:
Shared_encoder = Encoder(window_size * num_features, hidden_size)
AE1 = AutoEncoder(window_size * num_features, hidden_size, Shared_encoder)
AE2 = AutoEncoder(window_size * num_features, hidden_size, Shared_encoder)

In [7]:
Shared_encoder.to(device)
AE1.to(device)
AE2.to(device)

AutoEncoder(
  (encoder): Encoder(
    (fc1): Linear(in_features=5100, out_features=2550, bias=True)
    (relu): ReLU(inplace=True)
    (fc2): Linear(in_features=2550, out_features=1275, bias=True)
    (fc3): Linear(in_features=1275, out_features=1000, bias=True)
  )
  (sigmoid): Sigmoid()
  (relu): ReLU(inplace=True)
  (fc1): Linear(in_features=1000, out_features=1275, bias=True)
  (fc2): Linear(in_features=1275, out_features=2550, bias=True)
  (fc3): Linear(in_features=2550, out_features=5100, bias=True)
)

In [8]:
optimizer1 = torch.optim.Adam(AE1.parameters())
optimizer2 = torch.optim.Adam(AE2.parameters())

In [9]:
AE1, AE2, AE1_history, AE2_history = train_USAD(AE1, AE2, optimizer1, optimizer2, train_loader, validation_loader, epochs)

Epoch: 0 ---> Val loss: AE1 0.4368, AE2: 0.4339
Train loss: AE1 0.8754, AE2 0.8694
Epoch: 1 ---> Val loss: AE1 0.4371, AE2: 0.0000
Train loss: AE1 0.8683, AE2 0.0000
Epoch: 2 ---> Val loss: AE1 0.4331, AE2: -0.1440
Train loss: AE1 0.8675, AE2 -0.2885
Epoch: 3 ---> Val loss: AE1 0.4333, AE2: -0.2163
Train loss: AE1 0.8667, AE2 -0.4327
Epoch: 4 ---> Val loss: AE1 0.4336, AE2: -0.2598
Train loss: AE1 0.8667, AE2 -0.5194
Epoch: 5 ---> Val loss: AE1 0.4343, AE2: -0.2892
Train loss: AE1 0.8662, AE2 -0.5769
Epoch: 6 ---> Val loss: AE1 0.4336, AE2: -0.3094
Train loss: AE1 0.8662, AE2 -0.6181
Epoch: 7 ---> Val loss: AE1 0.4334, AE2: -0.3247
Train loss: AE1 0.8667, AE2 -0.6495
Epoch: 8 ---> Val loss: AE1 0.4342, AE2: -0.3374
Train loss: AE1 0.8657, AE2 -0.6728
Epoch: 9 ---> Val loss: AE1 0.4340, AE2: -0.3470
Train loss: AE1 0.8661, AE2 -0.6925
Epoch: 10 ---> Val loss: AE1 0.4348, AE2: -0.3555
Train loss: AE1 0.8659, AE2 -0.7081
Epoch: 11 ---> Val loss: AE1 0.4331, AE2: -0.3608
Train loss: AE1 0.

KeyboardInterrupt: 

In [ ]:
def plot_history(losses1, losses2):
    plt.plot(losses1, '-x', label="loss1")
    plt.plot(losses2, '-x', label="loss2")
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend()
    plt.title('Losses vs. No. of epochs')
    plt.grid()
    plt.savefig('Training_history.png')
    plt.show()

In [ ]:
plot_history(AE1_history, AE2_history)

In [ ]:
# AE1.load_state_dict(torch.load('my_USAD_without_downsampling_AE1_hidden_10_random.pth'))
# AE2.load_state_dict(torch.load('my_USAD_without_downsampling_AE2_hidden_10_random.pth'))

In [ ]:
test_loader, y_test = get_test_loader(columns, batch_size, window_size, device, mm)

In [ ]:
results = testing(AE1, AE2, test_loader)
y_pred=np.concatenate([torch.stack(results[:-1]).flatten().detach().cpu().numpy(),
                       results[-1].flatten().detach().cpu().numpy()])
threshold = ROC(y_test, y_pred)

In [ ]:
threshold

In [ ]:
ff = 0
f_history = []
initial_y = 7
best = -1
for inc in np.linspace(0, 7, 40):
    y_pred_int = [1 if (yy > initial_y + inc) else 0 for yy in y_pred]
    conf_matrix = confusion_matrix(y_test, y_pred_int)
    TP = conf_matrix[1, 1]
    TN = conf_matrix[0, 0]
    FP = conf_matrix[0, 1]
    FN = conf_matrix[1, 0]
    F1 = f1_score(y_test, y_pred_int)
    f_history.append(F1)

    precision = precision_score(y_test, y_pred_int)
    recall = recall_score(y_test, y_pred_int)
    if F1 > ff:
        ff = F1
        best = inc
print(f'best: {best + initial_y} F1: {ff}')
plt.plot(f_history)